## First, we would like to create a function that randomly samples the dataset for onsets

# NOTE: Run the next two cells to generate 'onsets' and 'songs', described below. These can be used to test onset detection methods

## second note: change "processed_path" to your own processed path, probably .../Training Data/Processed

In [13]:
from pathlib import Path
import os

# Generate a list of simplified notes files
processed_path = Path(r'/Users/ewaissbluth/Documents/GitHub/tensor-hero/Training Data/Training Data/Processed')
song_paths = [] # paths to song.ogg files
notes_simplified_paths = [] # paths to corresponding notes_simplified
for p1 in [processed_path / x for x in os.listdir(processed_path)]:
    if '.DS_Store' in str(p1):
        continue
    for p2 in [p1 / x for x in os.listdir(p1)]:
        if '.DS_Store' in str(p2):
            continue
        if 'notes_simplified.npy' in os.listdir(p2):
            song_path = Path(str(p2).replace('Processed', 'Unprocessed')) / 'song.ogg'
            if not os.path.exists(song_path) == True:
                continue
            notes_simplified_paths.append(p2 / 'notes_simplified.npy')
            song_paths.append(song_path)

641


In [34]:
import random
import numpy as np
import librosa
import math
import pickle
from tqdm import tqdm

# Select 20 songs at random, from each song, select one 60 second slice
subset_idx = random.sample([x for x in range(len(song_paths))], 20)
subset_idx.sort()

# Define empty lists for the songs and the onsets
# songs = [<numpy array song 1>, <numpy array song 2> ...]
# onsets = [<onset times song 1>, <onset times song 2> ...]
songs = []
onsets = []
# Load the songs and the notes arrays one at a time
for idx in tqdm(subset_idx):
    # Load the song
    y, sr = librosa.load(song_paths[idx])
    
    # resample the song if it isn't sr=22050 (for consistent sizing)
    if not sr == 22050:
        y = librosa.resample(y, sr, 22050)
        sr = 22050
    
    # Load notes
    notes = np.load(notes_simplified_paths[idx])
    # Cut off the appended silence from the notes
    notes = notes[7:-7]

    # Choose a random 60 second subset
    song_len_in_seconds = math.floor(y.shape[0]/sr)
    random_point = random.randint(0, song_len_in_seconds-60)

    # slice the song and the notes
    subsong = y[random_point*sr:(random_point+60)*sr]
    subnotes = notes[random_point*100:(random_point+60)*100]

    # Append the subsong to the array
    songs.append(subsong)

    # Get onsets from notes
    notes_times = np.where(subsong > 0)[0] / 100

    # Append onsets to array
    onsets.append(notes_times)

# Save songs and onsets
with open('onsets.pkl', 'wb') as f:
    pickle.dump(onsets, f)
f.close()
with open('songs.pkl', 'wb') as f:
    pickle.dump(songs, f)
f.close()

## Next, we would like to define the skeleton of some code that can be used to evaluate the different onset detection methods

In [ ]:
# First, we load onsets and song segments. Recall, "onsets" contains
# ground truth onsets that we calculated from simplified_notes.npy
# songs contains numpy arrays of song waveforms.
with open('onsets.pkl', 'rb') as f:
    onsets = pickle.load(f)
f.close()
with open('songs.pkl', 'rb') as f:
    songs = pickle.load(f)
f.close()

# PSEUDOCODE:
# Next, we iterate through onsets and songs
for i in range(len(onsets)):
    reference_onset = onsets[i]
    audio = songs[i]

    # DO ANALYSIS ON EACH SONG SEGMENT
    # REPORT THE AVERAGE F1 SCORE OVER ALL 20 SEGMENTS FOR EACH
    # HYPERPARAMETER / PREPROCESSING METHOD
